In [1]:
import cv2
import numpy as np
import cv2 as cv
import glob

In [4]:
################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (9,6)
frameSize = (640,480)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)
objp = objp*23

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.


imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')

for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imshow('img right', imgR)
        cv.waitKey(1000)


cv.destroyAllWindows()

In [5]:
#### Creating the A matrices

matricesLeft = []
matricesRight = []

for i,obj in enumerate(objpoints):
    A_left = []
    A_right = []
    for n,point in enumerate(obj):
        A_left.append([-point[0],-point[1],-1,0,0,0,imgpointsL[i][n][0][0]*point[0],imgpointsL[i][n][0][0]*point[1],imgpointsL[i][n][0][0]])
        A_left.append([0,0,0,-point[0],-point[1],-1,imgpointsL[i][n][0][1]*point[0],imgpointsL[i][n][0][1]*point[1],imgpointsL[i][n][0][1]])
        
        A_right.append([-point[0],-point[1],-1,0,0,0,imgpointsR[i][n][0][0]*point[0],imgpointsR[i][n][0][0]*point[1],imgpointsR[i][n][0][0]])
        A_right.append([0,0,0,-point[0],-point[1],-1,imgpointsR[i][n][0][1]*point[0],imgpointsR[i][n][0][1]*point[1],imgpointsR[i][n][0][1]])
    
    matricesLeft.append(np.array(A_left))
    matricesRight.append(np.array(A_right))
    
###### Computing B:
V_left = []
V_right = []
for i in range(len(matricesLeft)):
    ATA_left = matricesLeft[i].T@matricesLeft[i]
    eig_left, vec_left = np.linalg.eig(ATA_left)
    #print(eig_left)
    M_left = vec_left.T[-1]
    m1 = [M_left[0],M_left[3],M_left[6]]
    m2 = [M_left[1],M_left[4],M_left[7]]
    V_left.append([m1[0]*m2[0],m1[1]*m2[0]+m1[0]*m2[1],m1[2]*m2[0]+m1[0]*m2[2],m1[1]*m2[1],m1[2]*m2[1]+m1[1]*m2[2],m1[2]*m2[2]])
    V_left.append([m1[0]**2-m2[0]**2,2*m1[1]*m1[0]-2*m2[1]*m2[0],2*m1[2]*m1[0]-2*m2[2]*m2[0],m1[1]**2-m2[1]**2,2*m1[2]*m1[1]-2*m2[2]*m2[1],m1[2]**2-m2[2]**2])
    
    ATA_right = matricesRight[i].T@matricesRight[i]
    eig_right, vec_left = np.linalg.eig(ATA_right)
    #print(eig_right)
    M_right = vec_left.T[-1]
    m1 = [M_right[0],M_right[3],M_right[6]]
    m2 = [M_right[1],M_right[4],M_right[7]]
    V_right.append([m1[0]*m2[0],m1[1]*m2[0]+m1[0]*m2[1],m1[2]*m2[0]+m1[0]*m2[2],m1[1]*m2[1],m1[2]*m2[1]+m1[1]*m2[2],m1[2]*m2[2]])
    V_right.append([m1[0]**2-m2[0]**2,2*m1[1]*m1[0]-2*m2[1]*m2[0],2*m1[2]*m1[0]-2*m2[2]*m2[0],m1[1]**2-m2[1]**2,2*m1[2]*m1[1]-2*m2[2]*m2[1],m1[2]**2-m2[2]**2])

    
    
V_left = np.array(V_left)
eigs, b_left = np.linalg.eig(V_left.T@V_left)
#print(eigs)
b_left = b_left.T[-1]
B_left = np.array([[b_left[0],b_left[1],b_left[2]],[b_left[1],b_left[3],b_left[4]],[b_left[2],b_left[4],b_left[5]]])
L_left = np.linalg.cholesky(B_left)
K_left = np.linalg.inv(L_left.T)/np.linalg.inv(L_left.T)[2,2]

    
V_right = np.array(V_right)
eigs, b_right = np.linalg.eig(V_right.T@V_right)
#print(eigs)
b_right = b_right.T[-1]
B_right = np.array([[b_right[0],b_right[1],b_right[2]],[b_right[1],b_right[3],b_right[4]],[b_right[2],b_right[4],b_right[5]]])
L_right = np.linalg.cholesky(B_right)
K_right = np.linalg.inv(L_right.T)/np.linalg.inv(L_right.T)[2,2]

print("K_left:")
print(K_left)
print("K_right:")
print(K_right)

[1.01826812e+11 9.81070060e+09 1.06533664e+06 7.47794893e+05
 1.10305089e+05 2.73304951e+04 7.96993095e+03 4.80345966e+00
 1.05687449e-05]
[9.37599232e+10 9.51084501e+09 1.11731710e+06 7.53484637e+05
 1.10261618e+05 2.98260729e+04 8.39920269e+03 5.16971010e+00
 8.91421859e-06]
[1.31345941e+11 1.24384773e+10 7.84801302e+05 8.42754171e+05
 1.23091678e+05 9.56337500e+03 3.67817888e+03 3.41173839e-05
 9.11879681e+00]
[1.28253444e+11 1.14152797e+10 8.07770682e+05 7.75599805e+05
 1.20585016e+05 9.88171389e+03 4.67771557e+03 8.11587723e+00
 3.22589225e-05]
[1.03922147e+11 1.08016462e+10 8.02807896e+05 7.08683396e+05
 1.19782420e+05 7.49174527e+03 4.97553150e+03 6.26419085e+00
 3.09566653e-05]
[9.94262400e+10 1.07404750e+10 7.86167592e+05 6.47606192e+05
 1.25207989e+05 9.81652682e+03 4.31030671e+03 6.27668710e+00
 6.70888315e-05]
K_left:
[[266.49139444  47.60967801 577.36281308]
 [  0.         267.02810358 410.11277805]
 [  0.           0.           1.        ]]
K_right:
[[6.78195930e+02 2.672

In [25]:
#### Recovering the Transformation matrices

